Antes hemos terminado con un modelo que parecía que tenía un serio sobreajuste. En este nuevo capítulo vamos a intentar resolver ese problema usando lo que aprendimos en el anterior capítulo.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.model_selection import train_test_split

from lib.datasets import load_extended_boston
from lib.plots import plot_ridge_n_samples

In [ ]:
X, y = load_extended_boston()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

## Ridge

La regresión *Ridge* o *Ridge regression* es un modelo lineal para regresión. Usa la misma fórmula que usa la regresión lineal pero los pesos (coeficientes) se eligen de tal forma que no solo predigan bien en los datos de entrenamiento sino que también han de tener en cuenta, ajustar, una pequeña limitación. Esto es lo que vimos como regularización L2. Además, queremos con la magnitud de los pesos sea tan pequeña como sea posible, cercanos a cero. Si pensamos sobre ello de forma intuitiva, ¿qué significa esto?

(\*)
<p style="color: white">
    Nuestros pesos tendrán poco peso sobre el resultado final, su pendiente será pequeña, pero seguirán siendo capaces de predecir bien.
<\p>

Vamos a hacer lo mismo que hicimos con la regresión lineal usando los mismos datos que, si os acordáis, estaba sobreestimando.

In [ ]:
linrid = Ridge().fit(X_train, y_train)

A ver cómo es su $R^2$ en los datos de entrenamiento:

In [ ]:
print(linrid.score(X_train, y_train))

Y en los datos de prueba:

In [ ]:
print(linrid.score(X_test, y_test))

Como podéis ver los valores son menor y mayor para los datos de entrenamiento y de prueba, respectivamente, cuando lo comparamos con el caso de la regresión lineal (0.952, 0.607) que vimos en el anterior capítulo. Recordad que la regresión lineal estaba sobreajustando.

Un modelo menos complejo es un modelo que funcionará un poco mejor en el entrenamiento pero generalizará mejor. Como estamos interesados en el rendimiento de la generalización deberíamos elegir el modelo *Ridge* en lugar del modelo de regresión lineal para este caso concreto.

Como expertos científicos de datos deberemos ajustar correctamente el parámetro $ \lambda $ de la regularización L2 que usa este modelo. Esta $ \lambda $ es el parámetro `alpha` en `scikit-learn`. En el anterior ejemplo hemos usado el valor por defecto que usa `scikit-learn` (`alpha=1`). El valor óptimo de `alpha` dependerá de los datos que usemos. Incrementarlo llevará a tener peor ajuste en el entrenamiento pero puede llevar a mejor generalización. Vamos a aprender a usarlo.

Incrementar `alpha` significa hacer que los pesos o coeficientes estén más restringidos.

In [ ]:
linrid10 = Ridge(alpha=10).fit(X_train, y_train)

In [ ]:
print(linrid10.score(X_train, y_train), linrid10.score(X_test, y_test))

En este caso generaliza peor que si usamos el valor por defecto, `alpha=1`.

¿Qué pasa si bajamos el valor de `alpha` por debajo del valor por defecto?

In [ ]:
linrid01 = Ridge(alpha=0.1).fit(X_train, y_train)

In [ ]:
print(linrid01.score(X_train, y_train), linrid01.score(X_test, y_test))

De momento no vamos a optimizar para el hiperparámetro $ \lambda $ (o `alpha` para el caso de `Ridge` en `scikit-learn`). El tema de ajustar los hiperparámetros y otras cosas lo dejamos para más adelante.

Vamos a traer de nuevo la regresión lineal para hacer comparaciones de sus coeficientes (los pesos):

In [ ]:
linreg = LinearRegression().fit(X_train, y_train)

Vamos a dibujar los coeficientes en función del modelo que estemos usando para poder compararlos:

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(linrid.coef_, 'ks', label="Ridge alpha=1")
ax.plot(linrid10.coef_, 'b*', label="Ridge alpha=10")
ax.plot(linrid01.coef_, 'yv', label="Ridge alpha=0.1")
ax.plot(linreg.coef_, 'ro', label="LinearRegression")
ax.set_xlabel("Coefficient index")
ax.set_ylabel("Coefficient magnitude")
ax.hlines(0, 0, len(linreg.coef_))
ax.set_ylim(-25, 25)
ax.legend()

En la anterior figura estamos viendo el valor de cada peso, $ w_i $, en cada modelo. En el modelo de regresión lineal veis que algunas *features* tienen valores muy altos (por arriba o positivos y por debajo o negativos, quitad la línea `ax.set_ylim(-25, 25)` y ved los coeficientes de forma más drástica). En el caso de *Ridge regression* cuando el modelo usa `alpha=0.1` vemos como algunso coeficientes pueden ser incluso más extremos que en el caso de la regresión lineal pero suelen estar más acotados. Los otros dos casos, `alpha=1` o `alpha=10`, se puede ver que los coeficientes estan cercanos a cero en la mayoría de los casos.

Otra forma de ver la influencia de la regularización sería fijando un valor de `alpha` pero usando diferente número de datos en el entrenamiento. Vamos a cerlo en acción y comentamos sobre el gráfico:

In [ ]:
plt.figure(figsize=(15, 5))
plot_ridge_n_samples()

El gráfico anterior se llama curva de aprendizaje (*learning curve*). Es un gráfico que muestra el rendimiento del modelo en función del tamaño de la muestra de datos.

Lo que vemos en el anterior gráfico es el valor de $ R^2 $ del modelo lineal y del `Ridge` con `alpha=1` (valor por defecto). Cosas interesantes que vemos:

(\*)
<div style="color: white">
* En general, como `Ridge` es regularizado obtenemos peores valores del *score* en el conjunto de datos de entrenamiento.
* Con muy poquitos datos la regresión lineal no aprende nada.
* Con una cantidad aceptable de datos el `Ridge` empieza a dar cosas aceptables.
* A medida que metemos más datos la regularización es menos importante y es más difícil que el modelo sobreajuste ya que tenemos cada vez más diversidad. Mirad como baja el valor del *score* a medida que metemos más datos en el caso de la regresión lineal.
* ...
<\div>

Para los escépticos, si usamos `alpha=0` estamos en el caso de regresión lineal:

In [ ]:
linrid0 = Ridge(alpha=0, normalize=True).fit(X_train, y_train)

fig, ax = plt.subplots()

ax.plot(linrid0.coef_, linreg.coef_, 'o')
#ax.set_xscale('log')
#ax.set_yscale('log')

In [ ]:
for c, cc in zip(linreg.coef_, linrid0.coef_):
    print(c - cc, c, cc)

## Lasso

Una alternativa a `Ridge` para regularizar sería `Lasso` (*Least Absolute Shrinkage and Selection Operator*). La regresión *Lasso* usa regresión de tipo L1. La regularización L1 permite que algunos valores de coeficientes sean exactamente 0 por lo que no se usan para nada en el ajuste. Este permite hacer selección de *features* permitiendo eliminar algunas de ellos y que nuestro modelo sea más explicable al tener menos dimensiones.

Vamos a hacer lo mismo que antes pero en este caso para el modelo de regrsión *Lasso*:

In [ ]:
linlas = Lasso().fit(X_train, y_train)

Veamos como se comporta con los datos de entrenamiento.

In [ ]:
print(linlas.score(X_train, y_train))

Buff, que horror.

Y sobre los datos de prueba:

In [ ]:
print(linlas.score(X_test, y_test))

Ninguna maravilla. Vamos a ver el número de *features* que está usando:

In [ ]:
print(np.sum(linlas.coef_ != 0))

Solo 4 en comparación con el `Ridge` o la regresión lineal que están usando más de 100.

Como en el caso de `Ridge`, `Lasso` también tiene su hiperparámetro, la $ \lambda $ que podemos toquetear, que, al igualque en el caso de `Ridge`, en `scikit-learn` se llama `alpha`. El valor por defecto vuelve a ser `alpha=1` y, por tanto, es lo que hemos usado en el ejemplo anterior. Vimos que ambas regularizaciones, cuando incrementabamos el valor de $ \lambda $ (`alpha` en `scikit-learn`) los valores tendían más a 0. En el caso que estamos viendo quizá sea mejor usar un valor entre 0 y 1 porque incrementar aun más `alpha` nos podría dar incluso peores resultados. Vamos a hacer eso:

In [ ]:
linlas001 = Lasso(alpha=0.01).fit(X_train, y_train)

La anterior advertencia nos indica que quizá debemos aumentar el número de iteraciones para que los valores converjan a una tolerancia aceptable. Vamos a hacer caso a los expertos:

In [ ]:
linlas001 = Lasso(alpha=0.01, max_iter=100_000).fit(X_train, y_train)

Ya no tenemos la advertencia. Vamos a ver qué tal se comporta ahora:

In [ ]:
print(linlas001.score(X_train, y_train))

In [ ]:
print(linlas001.score(X_test, y_test))

In [ ]:
print(np.sum(linlas001.coef_ != 0))

Esto ya es otra cosa. El modelo es más complejo pero parece mucho más útil que nuestro intento inicial.

Nuevamente, si nos acercamos mucho a cero con el valor de `alpha` veremos que obtenemos valores parecidos a la regresión lineal puesto que la regularización es cada vez menos importante:

In [ ]:
linlas00001 = Lasso(alpha=0.0001, max_iter=100_000).fit(X_train, y_train)

In [ ]:
print(linlas00001.score(X_train, y_train))

In [ ]:
print(linlas00001.score(X_test, y_test))

In [ ]:
print(np.sum(linlas00001.coef_ != 0))

Al igual que antes, vamos a dibujar los valores de los coeficientes:

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))

ax.plot(linlas.coef_, 'ks', label="Lasso alpha=1")
ax.plot(linlas001.coef_, 'b^', label="Lasso alpha=0.01")
ax.plot(linlas00001.coef_, 'yv', label="Lasso alpha=0.0001")
ax.plot(linrid01.coef_, 'ro', label="Ridge alpha=0.1")
ax.legend(ncol=2, loc=(0, 1.05))
ax.set_ylim(-25, 25)
ax.set_xlabel("Coefficient index")
ax.set_ylabel("Coefficient magnitude")

¿Qué veis en la figura anterior?

...

**[INCISO]** `Lasso` (y `ElasticNet`) no usan gradiente descendiente para minimizar su función de coste. Esto es debido a que $ |\textbf{w}| $ no es diferenciable y no podemos usar el mismo algoritmo (gradiente descendiente). En este caso usan un algoritmo de optimización que se llama [*Coordinate descent*](https://en.wikipedia.org/wiki/Coordinate_descent).

## Cuando elegir entre `Ridge` y `Lasso`

En la práctica, *ridge regression* suele ser la primera elección entre estos dos modelos. Sin embargo, si tenemos una gran cantidad de dimensiones (o *features* o rasgos) y esperamos que solo unas pocas tengan importancia quizá la regresión *lasso* sea una buena opción. Otra opción para elegir *lasso* es cuando queremos interpretar el modelo de forma más sencilla.

## ElasticNet

`scikit-learn` (y muchas otras bibliotecas) ofrecen modelos conocidos como *ElasticNet* que combina las penalizaciones de la regularización L1 y de la regularización L2 y, por tanto, hemos de ajustar dos parámetros. En la práctica, esta combinación funciona mejor.

![ElasticNet loss function](./imgs/12_elasticnet_loss_function.png)

[Fuente](https://hackernoon.com/an-introduction-to-ridge-lasso-and-elastic-net-regression-cca60b4b934f?gi=1811236dc86f)

En la fórmula anterior vemos que tenemos dos parámetros, $ \lambda $ y $ \alpha $. Cuando $ \alpha $ vale 0 tenemos una regularización L2 mientras que si $ \alpha $ vale 1 tenemos una regularización L2.